In [2]:
import pandas as pd
from google.colab import files
files.upload()
files.upload()

Saving AD-HC151223.csv to AD-HC151223.csv


Saving classifier.py to classifier.py


{'classifier.py': b"import torch.nn as nn\nclass Normalize(nn.Module):\n\n    def __init__(self, power=2):\n        super(Normalize, self).__init__()\n        self.power = power\n\n    def forward(self, x):\n        norm = x.pow(self.power).sum(1, keepdim=True).pow(1. / self.power)\n        out = x.div(norm)\n        return out\nclass Flatten(nn.Module):\n    def __init__(self):\n        super(Flatten, self).__init__()\n\n    def forward(self, feat):\n        return feat.view(feat.size(0), -1)\n\nclass LinearClassifierAlexNet(nn.Module):\n    def __init__(self, in_dim, n_hid=200, n_label=3):\n        super(LinearClassifierAlexNet, self).__init__()\n        \n        self.classifier = nn.Sequential()\n        self.classifier.add_module('Flatten', Flatten())\n        #self.classifier.add_module('Dropout', nn.Dropout(p=0.3))\n        self.classifier.add_module('LinearClassifier', nn.Linear(in_dim, n_hid))\n        #self.classifier.add_module('ReLU', nn.ReLU(inplace=True))\n        \n     

In [3]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import altair as alt


df = pd.read_csv('AD-HC151223.csv').dropna()
df.head()



,Diagnosis,SubjectID,Feature_1,Feature_2,Feature_3,Feature_4,Feature_5,Feature_6,Feature_7,Feature_8,...,Feature_1015,Feature_1016,Feature_1017,Feature_1018,Feature_1019,Feature_1020,Feature_1021,Feature_1022,Feature_1023,Feature_1024
0,ea,249,0.225644,0.294361,-0.586159,0.092565,0.279320,0.319655,0.353205,0.796064,...,-0.407772,0.128206,0.483690,-1.037158,0.253173,-0.478867,-0.005794,0.428884,0.215939,-0.047832
1,tb,340,0.176566,0.003922,-0.822623,0.142225,0.339827,0.368419,0.077795,0.675404,...,-0.330383,0.359157,0.225034,-0.880441,0.330383,-0.486661,0.084016,0.497953,0.272045,-0.005895
2,crl,148,0.059418,0.249234,-0.629175,0.119050,0.287834,0.231810,0.108939,1.000264,...,-0.477618,0.216262,0.308836,-0.828842,0.191738,-0.423216,0.014898,0.568146,0.480394,-0.045237
3,tb,330,0.120042,0.182887,-0.631385,0.071078,0.407276,0.314706,0.091156,0.686448,...,-0.467419,0.257020,0.279692,-0.896820,0.228815,-0.307612,-0.040840,0.504867,0.175587,0.024474
4,ea,229,0.216866,0.125677,-0.494574,-0.080123,0.450195,0.283400,0.322392,0.991242,...,-0.447377,0.120799,0.399428,-0.662084,0.492690,-0.387376,-0.157657,0.674149,0.370550,0.027543


In [4]:
diagnoses = df['Diagnosis']
features = df.drop(columns=['Diagnosis', 'SubjectID'])

scaler = StandardScaler()
scaled_features = scaler.fit_transform(features)



pca = PCA(n_components=0.95)
pca_result = pca.fit_transform(scaled_features)


n_tsne_components = 2

tsne = TSNE(n_components=n_tsne_components, perplexity=50, n_iter=600)
tsne_result = tsne.fit_transform(pca_result)


tsne_df = pd.DataFrame(data=tsne_result, columns=['Dimension 1', 'Dimension 2']).reset_index(drop=True)
tsne_df['Diagnosis'] = diagnoses



scatter = alt.Chart(tsne_df).mark_circle().encode(
    x='Dimension 1',
    y='Dimension 2',
    color='Diagnosis:N',
    tooltip=['Diagnosis']
).properties(
    width=600,
    height=400
)
scatter


alt.Chart(...)

In [5]:
files.upload()

Saving classifier.py to classifier (1).py


{'classifier (1).py': b"import torch.nn as nn\nclass Normalize(nn.Module):\n\n    def __init__(self, power=2):\n        super(Normalize, self).__init__()\n        self.power = power\n\n    def forward(self, x):\n        norm = x.pow(self.power).sum(1, keepdim=True).pow(1. / self.power)\n        out = x.div(norm)\n        return out\nclass Flatten(nn.Module):\n    def __init__(self):\n        super(Flatten, self).__init__()\n\n    def forward(self, feat):\n        return feat.view(feat.size(0), -1)\n\nclass LinearClassifierAlexNet(nn.Module):\n    def __init__(self, in_dim, n_hid=200, n_label=3):\n        super(LinearClassifierAlexNet, self).__init__()\n        \n        self.classifier = nn.Sequential()\n        self.classifier.add_module('Flatten', Flatten())\n        #self.classifier.add_module('Dropout', nn.Dropout(p=0.3))\n        self.classifier.add_module('LinearClassifier', nn.Linear(in_dim, n_hid))\n        #self.classifier.add_module('ReLU', nn.ReLU(inplace=True))\n        \n 

In [80]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.decomposition import PCA
import pandas as pd
import numpy as np
import random
from classifier import LinearClassifierAlexNet, AdversarialClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import resample
from sklearn.utils.class_weight import compute_class_weight
from sklearn.model_selection import StratifiedKFold
from scipy.stats import sem, t

# Set seeds for reproducibility
seed = 42
np.random.seed(seed)
torch.manual_seed(seed)


df = pd.read_csv('AD-HC151223.csv')
df = df[df['Diagnosis']!='tb'] #leaves only 54 rows


Y = df['Diagnosis']
X = df.drop(columns=['Diagnosis', 'SubjectID'])  # Drop non-feature columns

label_encoder = LabelEncoder()
Y_encoded = label_encoder.fit_transform(Y)

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

#pca = PCA(n_components=0.99)
#X_pca = pca.fit_transform(X_scaled)

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X_scaled, Y_encoded, test_size=0.5, random_state=seed)

# Convert to  tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)
y_test = torch.tensor(y_test, dtype=torch.long)

unique_labels = np.unique(y_train.numpy())
print("Unique labels in y_train:", unique_labels)

# make dataloaders
train_loader = DataLoader(TensorDataset(X_train, y_train), batch_size=64, shuffle=True)
test_loader = DataLoader(TensorDataset(X_test, y_test), batch_size=64)

# init models
model1 = LinearClassifierAlexNet(X_train.shape[1], n_hid = 200, n_label= 2)
model2 = AdversarialClassifier(X_train.shape[1], 200, 2)

#Try to address class imbalance:
y_train_np = y_train.numpy() if isinstance(y_train, torch.Tensor) else np.array(y_train) #troubleshoot
class_weights = compute_class_weight('balanced', classes=np.unique(y_train.numpy()), y=y_train.numpy())
class_weights_tensor = torch.tensor(class_weights, dtype=torch.float32)

# make sure the class weights tensor length matches the number of classes
print("Class weights:", class_weights_tensor)

# Function to train a model
def train_model(model, train_loader, class_weights_tensor):
    criterion = nn.CrossEntropyLoss(weight=class_weights_tensor)
    optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

    for epoch in range(100):
        model.train()
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

# Function to evaluate a model
def evaluate_model(model, test_loader):
    model.eval()
    predictions, labels = [], []
    with torch.no_grad():
        for inputs, labels_batch in test_loader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            predictions.extend(predicted.cpu().numpy())
            labels.extend(labels_batch.cpu().numpy())

    accuracy = accuracy_score(labels, predictions)
    precision = precision_score(labels, predictions)
    recall = recall_score(labels, predictions)
    f1 = f1_score(labels, predictions)
    auc = roc_auc_score(labels, predictions)


    return accuracy, precision, recall, f1, auc

# Train and evaluate LinearClassifierAlexNet
train_model(model1, train_loader, class_weights_tensor)
acc1, prec1, rec1, f11, auc1 = evaluate_model(model1, test_loader)
print(f"LinearClassifierAlexNet - Accuracy: {acc1}, Precision: {prec1}, Recall: {rec1}, F1-Score: {f11}, AUC: {auc1}")

# Train and evaluate AdversarialClassifier
train_model(model2, train_loader, class_weights_tensor)
acc2, prec2, rec2, f12, auc2 = evaluate_model(model2, test_loader)
print(f"AdversarialClassifier - Accuracy: {acc2}, Precision: {prec2}, Recall: {rec2}, F1-Score: {f12}, AUC: {auc2}")

Unique labels in y_train: [0 1]
Class weights: tensor([1.9286, 0.6750])
LinearClassifierAlexNet - Accuracy: 0.5185185185185185, Precision: 0.6666666666666666, Recall: 0.26666666666666666, F1-Score: 0.3809523809523809, AUC: 0.55
AdversarialClassifier - Accuracy: 0.5185185185185185, Precision: 0.625, Recall: 0.3333333333333333, F1-Score: 0.43478260869565216, AUC: 0.5416666666666666


In [81]:

#Calculate confidence intervals


def bootstrap_auc(y_true, y_pred, n_bootstraps=1000, ci=95):
    bootstrapped_scores = []

    for _ in range(n_bootstraps):
        # Random sampling with replacement
        indices = resample(np.arange(len(y_pred)), replace=True)
        if len(np.unique(y_true[indices])) < 2:
            continue  # If there isn't both classes in the sampled indices, skip this iteration

        score = roc_auc_score(y_true[indices], y_pred[indices])
        bootstrapped_scores.append(score)

    sorted_scores = np.array(bootstrapped_scores)
    sorted_scores.sort()

    lower = np.percentile(sorted_scores, (100 - ci) / 2)
    upper = np.percentile(sorted_scores, 100 - (100 - ci) / 2)

    return lower, upper

# to get model predictions:
def get_model_predictions(model, test_loader):
    model.eval()
    all_labels = []
    all_probs = []
    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            probabilities = torch.nn.functional.softmax(outputs, dim=1)[:, 1].cpu().numpy()
            all_labels.extend(labels.cpu().numpy())
            all_probs.extend(probabilities)
    return all_labels, all_probs

#get predictions from the models
labels1, probs1 = get_model_predictions(model1, test_loader)
labels2, probs2 = get_model_predictions(model2, test_loader)

# AUC with confidence intervals
auc_ci_lower1, auc_ci_upper1 = bootstrap_auc(np.array(labels1), np.array(probs1))
auc_ci_lower2, auc_ci_upper2 = bootstrap_auc(np.array(labels2), np.array(probs2))

print(f"LinearClassifierAlexNet AUC: {auc1} (95% CI: {auc_ci_lower1}-{auc_ci_upper1})")
print(f"AdversarialClassifier AUC: {auc2} (95% CI: {auc_ci_lower2}-{auc_ci_upper2})")


LinearClassifierAlexNet AUC: 0.55 (95% CI: 0.5-0.9285714285714286)
AdversarialClassifier AUC: 0.5416666666666666 (95% CI: 0.40902847152847155-0.8516534391534393)


In [82]:
def cross_validated_auc(model_class, X, y, n_splits=5, **model_kwargs):
    cv = StratifiedKFold(n_splits=n_splits)
    auc_scores = []

    for train_idx, test_idx in cv.split(X, y):
        X_train, X_test = X[train_idx], X[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]

        # Initialize a new instance of the model for each fold
        model = model_class(**model_kwargs)

        # Convert to tensors
        X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
        y_train_tensor = torch.tensor(y_train, dtype=torch.long)
        X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
        y_test_tensor = torch.tensor(y_test, dtype=torch.long)


        train_loader = DataLoader(TensorDataset(X_train_tensor, y_train_tensor), batch_size=4, shuffle=True)

        # Train the model
        train_model(model, train_loader, class_weights_tensor)  # Ensure class weights are used

        # Evaluate the model
        test_loader = DataLoader(TensorDataset(X_test_tensor, y_test_tensor), batch_size=4)
        labels, probs = get_model_predictions(model, test_loader)
        auc = roc_auc_score(labels, probs)
        auc_scores.append(auc)

    # mean and standard error of the mean
    auc_mean = np.mean(auc_scores)
    auc_sem = sem(auc_scores)

    # Calculate the confidence interval
    confidence_interval = t.interval(0.95, len(auc_scores) - 1, loc=auc_mean, scale=auc_sem)

    return auc_mean, confidence_interval


mean_auc, ci_auc = cross_validated_auc(LinearClassifierAlexNet, X_scaled, Y_encoded, in_dim=X_train.shape[1], n_hid=200, n_label=2)
print(f"Linear Classifier: Mean AUC: {mean_auc}, 95% Confidence Interval: {ci_auc}")


mean_auc, ci_auc = cross_validated_auc(AdversarialClassifier, X_scaled, Y_encoded, in_dim=X_train.shape[1], nhid = 200, out_dim =2)
print(f"Adversarial Classifier: Mean AUC: {mean_auc}, 95% Confidence Interval: {ci_auc}")


Linear Classifier: Mean AUC: 0.7571428571428571, 95% Confidence Interval: (0.5553846405340954, 0.9589010737516188)
Adversarial Classifier: Mean AUC: 0.6976190476190476, 95% Confidence Interval: (0.4076562143580895, 0.9875818808800056)
